## 策略简介

因子：样例因子（18个）

标注：未来5日涨幅分类，涨幅靠前的为1，涨幅靠后的为0

算法：决策树算法

类型：分类问题

训练集：10-15年

测试集：15-19年

选股依据：根据上涨概率值排序买入

持股数：30

持仓天数：5

In [20]:
# 本代码由可视化策略环境自动生成 2019年5月12日 00:01
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m6_run_bigquant_run(input_1, input_2, input_3):
    train_df = input_1.read()
    features = input_2.read()
    feature_min = train_df[features].quantile(0.005)
    feature_max = train_df[features].quantile(0.995)
    train_df[features] = train_df[features].clip(feature_min,feature_max,axis=1)
    data_1 = DataSource.write_df(train_df)
    test_df = input_3.read()
    test_df[features] = test_df[features].clip(feature_min,feature_max,axis=1)
    data_2 = DataSource.write_df(test_df)
    return Outputs(data_1=data_1, data_2=data_2, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m6_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：每日数据处理函数，每天执行一次
def m11_handle_data_bigquant_run(context, data):
    
    context.extension['index'] += 1
    # 不在换仓日就return,相当于后面的代码只会一个月运行一次，买入的股票会持有1个月
    if  context.extension['index'] % context.rebalance_days != 0:
        return 
    
    # 当前的日期
    date = data.current_dt.strftime('%Y-%m-%d')
    
    cur_data = context.indicator_data[context.indicator_data['date'] == date]
    # 根据日期获取调仓需要买入的股票的列表
    #stock_to_buy = list(cur_data.instrument[:context.stock_num])
    cur_data = cur_data[cur_data['pred_label'] == 1.0]
    
    stock_to_buy =  list(cur_data.sort_values('classes_prob_1.0',ascending=False).instrument)[:context.stock_num]
    if date == '2017-02-06':
        print(date, len(stock_to_buy), stock_to_buy)
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = [equity.symbol for equity in context.portfolio.positions]
    # 继续持有的股票：调仓时，如果买入的股票已经存在于目前的持仓里，那么应继续持有
    no_need_to_sell = [i for i in stock_hold_now if i in stock_to_buy]
    # 需要卖出的股票
    stock_to_sell = [i for i in stock_hold_now if i not in no_need_to_sell]
  
    # 卖出
    for stock in stock_to_sell:
        # 如果该股票停牌，则没法成交。因此需要用can_trade方法检查下该股票的状态
        # 如果返回真值，则可以正常下单，否则会出错
        # 因为stock是字符串格式，我们用symbol方法将其转化成平台可以接受的形式：Equity格式

        if data.can_trade(context.symbol(stock)):
            # order_target_percent是平台的一个下单接口，表明下单使得该股票的权重为0，
            #   即卖出全部股票，可参考回测文档
            context.order_target_percent(context.symbol(stock), 0)
    
    # 如果当天没有买入的股票，就返回
    if len(stock_to_buy) == 0:
        return

    # 等权重买入 
    weight =  1 / len(stock_to_buy)
    
    # 买入
    for stock in stock_to_buy:
        if data.can_trade(context.symbol(stock)):
            # 下单使得某只股票的持仓权重达到weight，因为
            # weight大于0,因此是等权重买入
            context.order_target_percent(context.symbol(stock), weight)
 
# 回测引擎：准备数据，只执行一次
def m11_prepare_bigquant_run(context):
    pass

# 回测引擎：初始化函数，只执行一次
def m11_initialize_bigquant_run(context):
    # 加载预测数据
  
    context.indicator_data = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))
    context.rebalance_days = 5
    context.stock_num = 30
    if 'index' not in context.extension:
        context.extension['index'] = 0
     

# 回测引擎：每个单位时间开始前调用一次，即每日开盘前调用一次。
def m11_before_trading_start_bigquant_run(context, data):
    pass


m1 = M.instruments.v2(
    start_date='2010-01-01',
    end_date='2015-01-01',
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m2 = M.advanced_auto_labeler.v2(
    instruments=m1.data,
    label_expr="""shift(close, -5) / shift(open, -1)-1
rank(label)
where(label>=0.75,1,where(label<=0.25, 0, NaN))""",
    start_date='',
    end_date='',
    benchmark='000300.SHA',
    drop_na_label=False,
    cast_label_int=False
)

m3 = M.input_features.v1(
    features="""(close_0-mean(close_0,12))/mean(close_0,12)*100
rank(std(amount_0,15))
rank_avg_amount_0/rank_avg_amount_8
ts_argmin(low_0,20)
rank_return_30
(low_1-close_0)/close_0
ta_bbands_lowerband_14_0
mean(mf_net_pct_s_0,4)
amount_0/avg_amount_3
return_0/return_5
return_1/return_5
rank_avg_amount_7/rank_avg_amount_10
ta_sma_10_0/close_0
sqrt(high_0*low_0)-amount_0/volume_0*adjust_factor_0
avg_turn_15/(turn_0+1e-5)
return_10
mf_net_pct_s_0
(close_0-open_0)/close_1
 """
)

m15 = M.general_feature_extractor.v7(
    instruments=m1.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m16 = M.derived_feature_extractor.v3(
    input_data=m15.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m7 = M.join.v3(
    data1=m2.data,
    data2=m16.data,
    on='date,instrument',
    how='inner',
    sort=False
)

m13 = M.dropnan.v1(
    input_data=m7.data
)

m9 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2015-01-01'),
    end_date=T.live_run_param('trading_date', '2019-04-24'),
    market='CN_STOCK_A',
    instrument_list='',
    max_count=0
)

m17 = M.general_feature_extractor.v7(
    instruments=m9.data,
    features=m3.data,
    start_date='',
    end_date='',
    before_start_days=0
)

m18 = M.derived_feature_extractor.v3(
    input_data=m17.data,
    features=m3.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m14 = M.dropnan.v1(
    input_data=m18.data
)

m6 = M.cached.v3(
    input_1=m13.data,
    input_2=m3.data,
    input_3=m14.data,
    run=m6_run_bigquant_run,
    post_run=m6_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m8 = M.RobustScaler.v13(
    train_ds=m6.data_1,
    features=m3.data,
    test_ds=m6.data_2,
    scale_type='standard',
    quantile_range_min=0.01,
    quantile_range_max=0.99,
    global_scale=True
)

m10 = M.decision_tree_classifier.v1(
    training_ds=m8.train_data,
    features=m3.data,
    predict_ds=m8.test_data,
    criterion='gini',
    feature_fraction=1,
    max_depth=30,
    min_samples_per_leaf=200,
    key_cols='date,instrument',
    other_train_parameters={}
)

m11 = M.trade.v4(
    instruments=m9.data,
    options_data=m10.predictions,
    start_date='',
    end_date='',
    handle_data=m11_handle_data_bigquant_run,
    prepare=m11_prepare_bigquant_run,
    initialize=m11_initialize_bigquant_run,
    before_trading_start=m11_before_trading_start_bigquant_run,
    volume_limit=0,
    order_price_field_buy='open',
    order_price_field_sell='close',
    capital_base=10000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2019-04-29 22:58:29.757352] INFO: bigquant: instruments.v2 开始运行..

[2019-04-29 22:58:29.795105] INFO: bigquant: 命中缓存

[2019-04-29 22:58:29.797167] INFO: bigquant: instruments.v2 运行完成[0.039815s].

[2019-04-29 22:58:29.800708] INFO: bigquant: advanced_auto_labeler.v2 开始运行..

[2019-04-29 22:58:29.827044] INFO: bigquant: 命中缓存

...

[2019-04-29 22:58:30.414610] INFO: bigquant: 命中缓存

[2019-04-29 22:58:30.416778] INFO: bigquant: decision_tree_classifier.v1 运行完成[0.066524s].

[2019-04-29 22:58:30.463574] INFO: bigquant: backtest.v8 开始运行..

[2019-04-29 22:58:30.493865] INFO: bigquant: 命中缓存

[2019-04-29 22:58:33.178611] INFO: bigquant: backtest.v8 运行完成[2.715025s].